In [2]:
# import modules 
import requests 
import pandas as pd 

# create a request header 
headers = {'User-Agent': "hbirring@seattleu.edu"}

# get all companies data
companyTickers = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)

# print(companyTickers.json()['0']['cik_str'])

companyCIK = pd.DataFrame.from_dict(companyTickers.json(), orient= 'index')

print(companyCIK)

companyCIK['cik_str']= companyCIK['cik_str'].astype(str).str.zfill(10)

print(companyCIK)

companyCIK = companyCIK.set_index('ticker')

alphabet_cik = companyCIK.at['GOOGL', 'cik_str']

print(alphabet_cik)

# get all companies data
companyFacts = requests.get(f"https://data.sec.gov/api/xbrl/companyfacts/CIK{alphabet_cik}.json", headers=headers)

print(companyFacts.json().keys())

       cik_str ticker                            title
0      1045810   NVDA                      NVIDIA CORP
1       789019   MSFT                   MICROSOFT CORP
2       320193   AAPL                       Apple Inc.
3      1018724   AMZN                   AMAZON COM INC
4      1652044  GOOGL                    Alphabet Inc.
...        ...    ...                              ...
10056  2048270  SMFRF  Sumitomo Forestry Co., Ltd./ADR
10057  2008315  ARHUF                       AAK AB/ADR
10058  2053411  PCPPF         PC Partner Group Ltd/ADR
10059  2051587  BSAAU    BEST SPAC I Acquisition Corp.
10060  1788707  ADYYF                   Adyen N.V./ADR

[10061 rows x 3 columns]
          cik_str ticker                            title
0      0001045810   NVDA                      NVIDIA CORP
1      0000789019   MSFT                   MICROSOFT CORP
2      0000320193   AAPL                       Apple Inc.
3      0001018724   AMZN                   AMAZON COM INC
4      0001652044  GOOGL

In [30]:
# Quick Ratio for GOOGL

import pandas as pd

# Load the data into a dataframe from the SEC EDGAR API response above ^^^^
assets_df = pd.DataFrame(companyFacts.json()['facts']['us-gaap']['AssetsCurrent']['units']['USD'])
liabilities_df = pd.DataFrame(companyFacts.json()['facts']['us-gaap']['LiabilitiesCurrent']['units']['USD'])
inventory_df = pd.DataFrame(companyFacts.json()['facts']['us-gaap']['InventoryNet']['units']['USD'])

# Tag each dataframe with a label to indenifty the data for a pivot table 
assets_df['label'] = 'assets'
liabilities_df['label'] = 'liabilities'
inventory_df['label'] = 'inventory'

#combine all dataframes into one
all_df = pd.concat([assets_df, liabilities_df, inventory_df], ignore_index=True)

# Filter the dataframe/response to 10-K only
filtered_df = all_df[(all_df['form'] == '10-K') & (all_df['fy'] == 2024)]

#pivot the dataframe to one row per accession number
pivot_df = filtered_df.pivot_table(index= ['accn', 'end'], columns= 'label', values='val', aggfunc='first').reset_index()

#calculate the quick ratio: (current assets - inventory) / current liabilities 
pivot_df['quick_ratio'] = (pivot_df.get('assets', 0) - pivot_df.get('inventory', 0)) / pivot_df.get('liabilities', 1)

#print out the quick ratio 
print(pivot_df[['end', 'quick_ratio']])

                        

label         end  quick_ratio
0      2023-12-31     2.096585
1      2024-12-31     1.836931


In [29]:
# Current Ratio for GOOGL
import pandas as pd

# Load the data into a dataframe from the SEC EDGAR API response above ^^^^
assets_df = pd.DataFrame(companyFacts.json()['facts']['us-gaap']['AssetsCurrent']['units']['USD'])
liabilities_df = pd.DataFrame(companyFacts.json()['facts']['us-gaap']['LiabilitiesCurrent']['units']['USD'])

#Label each DataFrame to distinquish them after pivoting 
assets_df['label'] = 'assets'
liabilities_df['label'] = 'liabilities'

#combine both into one dataframes
all_df = pd.concat([assets_df, liabilities_df], ignore_index=True)

# Filter for 10-K filings and fiscal year 2024 
filtered_df = all_df[(all_df['form'] == '10-K') & (all_df['fy'] == 2024)]

#pivot to get one row per accession number and end date 
pivot_df = filtered_df.pivot_table(index= ['accn', 'end'], columns= 'label', values='val', aggfunc='first').reset_index()

#calculate the current ratio = current assets / current liabilities 
pivot_df['current_ratio'] = pivot_df.get('assets', 0) / pivot_df.get('liabilities', 1)

# Display results
print(pivot_df[['end', 'current_ratio']])



label         end  current_ratio
0      2023-12-31       2.096585
1      2024-12-31       1.836931
